## Run Geodyn

In [ ]:
# %load_ext autoreload
# %autoreload 2

# import sys  
# import pickle 
# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn

# models_list = ['msis2'] 

# run = {}
# for i,val in enumerate( models_list ): #,'jaachia71']):# 'msis86','msis00',
#     run_settings = '/data/geodyn_proj/validation/NoCD_Arc24_AllDensModels_settings/NoCdArc24_TESTOrbStep_'+val+'.yaml'

#     ### Load the data into an object
#     run[val] = Pygeodyn(run_settings)
#     run[val].RUN_GEODYN()


## Load the Data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

models_list = ['msis2']

Obj_Geodyn = {}
for i,val in enumerate( models_list ): #,'jaachia71']):# 'msis86','msis00',
    run_settings = '/data/geodyn_proj/validation/NoCD_Arc24_AllDensModels_settings/NoCdArc24_'+val+'.yaml'
#     run_settings = '/data/geodyn_proj/validation/NoCD_Arc24_AllDensModels_settings/NoCdArc24_TESTOrbStep_'+val+'.yaml'

    ### Load the data into an object
    Obj_Geodyn[val] = Pygeodyn(run_settings)
    Obj_Geodyn[val].getData()



## Validation Stats and Free Space

In [ ]:
%load_ext autoreload
%autoreload 2

for i,val in enumerate( models_list ):
    run_settings = '/data/geodyn_proj/validation/NoCD_Arc24_AllDensModels_settings/NoCdArc24_'+val+'.yaml'
    ### Load the data into an object
    Obj_Geodyn[val].post_getData_ValidationMetrics()


In [ ]:
# Obj_Geodyn['msis2'].__dict__['CombinedOrbitsDF']['2018.314']

In [ ]:
# Obj_Geodyn['msis2'].__dict__['Trajectory_orbfil']['2018.314']['data_record']

In [ ]:
# Obj_Geodyn['msis2'].__dict__['Residuals_obs']['2018.314']

### Stats Function

In [ ]:
def STATS_residuals(residuals):
    import numpy as np
    n = np.size(residuals)
    mean = (1/n)*(np.sum(residuals))
    variance = (1/n)*(np.sum(np.square(residuals)))
    rms = np.sqrt(variance)
    rms_about_zero = np.sqrt((n/(n-1))*variance)
    return(mean,rms,rms_about_zero)


## Plotting Config.

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd

import numpy as np

config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

# Simplify Plotting Schemes:
col1 = px.colors.qualitative.Plotly[0]
col2 = px.colors.qualitative.Plotly[1]
col3 = px.colors.qualitative.Plotly[2]
col4 = px.colors.qualitative.Plotly[3]
col5 = px.colors.qualitative.Plotly[4]
col6 = px.colors.qualitative.Plotly[5]

## Plot Residuals with RMS

In [ ]:
obj_m1=Obj_Geodyn['msis2']


In [ ]:
import datetime
doy_dates = []
for i in obj_m1.__dict__['global_params']['arc_input']:
    doy_dates.append(datetime.datetime(int(i.split('.')[0]), 1, 1) + datetime.timedelta(int(i.split('.')[1]) - 1))

### XYZ residuals

In [ ]:
%load_ext autoreload
%autoreload 2


# obj_m1=Obj_Geodyn['msis2']


fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(['X', 'Y', 'Z']),
    vertical_spacing = 0.2)

comp_resids = {}

model_m1 = obj_m1.__dict__['global_params']['den_model']
for iarc,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):

    data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']


#     fig.update_layout(title=''.join(coords)+' Orbit Residuals',
#                     autosize=True,
#                     font=dict(size=14),
#                     legend= {'itemsizing': 'constant'})
    comp_resids[arc] = {}

    for i,val in enumerate(['X','Y','Z']):
        comp_resids[arc][val] = data_resids[val]
        
        (mean,rms,rms_about_zero) = STATS_residuals(comp_resids[arc][val])

        fig.add_trace(go.Scattergl(
                             x=obj_m1.__dict__['OrbitResids'][arc]['resids']['Date'],
                             y=data_resids[val],
                         name=val,
                         mode='markers',
                         marker=dict(
                             color='blue',
                             size=5,
                                    ),
                         showlegend=False),
                         secondary_y=False,
                         row=i+1, col=1)
        fig.add_trace(go.Scatter(
            x=[doy_dates[iarc]+ datetime.timedelta(hours=12)],
            y=[np.max(data_resids[val])],
#             xref="x domain",
#             yref="y domain",
            text=[str(np.round(rms_about_zero,3)) ,
                 ],
            mode="text",
        textfont = { 'color': ["red"]},
            showlegend=False,
            ),
             row=i+1, col=1)

fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=3, col=1)
# fig.update_xaxes( title="Date", row=1, col=1)


fig.show(config=config)

RMS_total_xyz = []

for iarc,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
    totalvec_xyz = []
    for icomp,val in enumerate(comp_resids[arc].keys()):
        print(arc, val)
        (mean,rms,rms_about_zero) = STATS_residuals(comp_resids[arc][val])
        print('    ','mean   ',':',mean)
        print('    ','rms    ',':',rms)
        print('    ','rms_0  ',':',rms_about_zero)
        totalvec_xyz.append(rms_about_zero)
    
    RMS_total_xyz.append(np.sqrt( ( np.square(totalvec_xyz[0]) + np.square(totalvec_xyz[1]) + np.square(totalvec_xyz[2]) ) /3  ))
print('    ','--------------------')
print('    ','RMS_total XYZ', RMS_total_xyz)
print('    ','--------------------')


### NTW residuals

In [ ]:
%load_ext autoreload
%autoreload 2


obj_m1=Obj_Geodyn['msis2']


fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(['N','T','W']),
    vertical_spacing = 0.2)

comp_resids = {}

model_m1 = obj_m1.__dict__['global_params']['den_model']
for iarc,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):

    data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']


#     state_vector_pce = np.transpose(np.array([obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['X'], 
#                                               obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Y'], 
#                                               obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Z'], 
#                                               obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Xdot'], 
#                                               obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Ydot'], 
#                                               obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Zdot']]))

#     NTW_pce  = [Convert_cartesian_to_NTW_returnall(x_pce) for x_pce in state_vector_pce]



#     state_vector_orbfil = np.transpose(np.array([ obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['X'], 
#                                                   obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Y'], 
#                                                   obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Z'], 
#                                                   obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Xdot'], 
#                                                   obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Ydot'], 
#                                                   obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Zdot']]))

#     NTW_orbfil  = [Convert_cartesian_to_NTW_returnall(x_orbfil) for x_orbfil in state_vector_orbfil]

    
    
    comp_resids[arc] = {}

    for i,val in enumerate(['N','T','W']):
#         comp_resids[arc][val] = np.array(NTW_pce)[:,i] - np.array(NTW_orbfil)[:,i]
        comp_resids[arc][val] = np.array(obj_m1.__dict__['OrbitResids'][arc]['data_PCE'][val]) - np.array(obj_m1.__dict__['OrbitResids'][arc]['data_orbfil'][val])
        
        (mean,rms,rms_about_zero) = STATS_residuals(comp_resids[arc][val])

        fig.add_trace(go.Scattergl(
                             x=data_resids['Date'],
                             y= data_resids[val],
                         name=val,
                         mode='markers',
                         marker=dict(
                             color='blue',
                             size=5,
                                    ),
                         showlegend=False),
                         secondary_y=False,
                         row=i+1, col=1)
        
        if val == 'T':
            y_plac=[np.min(comp_resids[arc][val])],
        else:
            y_plac=[np.max(comp_resids[arc][val])],

        fig.add_trace(go.Scatter(
            x=[doy_dates[iarc]+ datetime.timedelta(hours=12)],
            y= [np.min(comp_resids[arc][val])],
            text=[str(np.round(rms_about_zero,3)) ,
                 ],
            mode="text",
        textfont = { 'color': ["red"]},
            showlegend=False,
            ),
             row=i+1, col=1)
fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=3, col=1)


fig.show(config=config)


RMS_total_ntw = []
for iarc,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
    totalvec_ntw = []
    for icomp,val in enumerate(comp_resids[arc].keys()):
        print(arc, val)
        (mean,rms,rms_about_zero) = STATS_residuals(comp_resids[arc][val])
        print('    ','mean   ',':',mean)
        print('    ','rms    ',':',rms)
        print('    ','rms_0  ',':',rms_about_zero)
        totalvec_ntw.append(rms_about_zero)
    
    RMS_total_ntw.append( np.sqrt( ( np.square(totalvec_ntw[0]) + np.square(totalvec_ntw[1]) + np.square(totalvec_ntw[2]) )/3  ))
print('    ','--------------------')
print('    ','RMS_total NTW', RMS_total_ntw)
print('    ','--------------------')


    

### Plot Total RMS timeseries

In [ ]:

def plot_TotalRMS_XYZ_NTW(fig, obj_m1, plot_num):

    if plot_num == 0:
        col = col1
        x_annot = 1.09
        y_annot1 = 1
        y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = 1.09
        y_annot1 = .8
        y_annot2 = .7
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = 1.09
        y_annot1 = .5
        y_annot2 = .5
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = 1.09
        y_annot1 = .2
        y_annot2 = .3
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = 1.09
        y_annot1 =  0 
        y_annot2 = .1
        col = col5
        m_size = 3.5
        
    import datetime
    doy_dates = []
    for i in obj_m1.__dict__['global_params']['arc_input']:
        doy_dates.append(datetime.datetime(int(i.split('.')[0]), 1, 1) + datetime.timedelta(int(i.split('.')[1]) - 1))
    
    text_xyz = []
    for itxt,txt in enumerate(obj_m1.__dict__['Statistics']['RMS_total_XYZ']):
        text_xyz.append(str(np.round(obj_m1.__dict__['Statistics']['RMS_total_XYZ'][itxt],3)))
    fig.add_trace(go.Scattergl(
                             x=doy_dates,
                             y=obj_m1.__dict__['Statistics']['RMS_total_XYZ'],
                         name='XYZ',
                         mode='markers+text+lines',
                         marker=dict(
                             symbol = 'circle-open',
                             color=col,
                             size=10,
                                    ),
                         line=dict(color=col, 
                             width=4,
                             dash='dash'),
                    text = text_xyz,
                    textposition = "top center",
                    textfont = {'color': [col]*(itxt+1)},
                         showlegend=False),
                         secondary_y=False,
                         row=1, col=1)

    text_ntw = []
    for itxt,txt in enumerate(obj_m1.__dict__['Statistics']['RMS_total_NTW']):
        text_ntw.append(str(np.round(obj_m1.__dict__['Statistics']['RMS_total_NTW'][itxt],3)))
    fig.add_trace(go.Scattergl(
                             x=doy_dates,
                             y=obj_m1.__dict__['Statistics']['RMS_total_NTW'],
                         name='NTW',
                         mode='markers+text+lines',
                         marker=dict(
                             color=col,
                             size=10,
                                    ),
                    text = text_ntw,
                    textposition = "top center",
                    textfont = {'color': [col]*(itxt+1)},
                         showlegend=False),
                         secondary_y=False,
                         row=1, col=1)

    return(fig)
    


In [ ]:
%load_ext autoreload
%autoreload 2

fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['Total RMS Values across multiple arcs']),
    vertical_spacing = 0.1)

fig = plot_TotalRMS_XYZ_NTW(fig,  Obj_Geodyn['msis2'] , 0 )

fig.show(config=config)


## Compare IIEOUT Resids against Manual Resids

In [ ]:
# def plot_residuals_manual_and_auto_compare(fig, obj_m1, plot_num):
# #     arc_list = [obj_m1.__dict__['run_parameters']['ARC']]
#     model_m1 = obj_m1.__dict__['global_params']['den_model']
# #     iter_list = [ obj_m1.__dict__['run_parameters']['str_iteration']]

#     if plot_num == 0:
#         col = col1
#         x_annot = 1.09
#         y_annot = .90
#         m_size = 4
#     elif plot_num == 1:
#         x_annot = 1.09
#         y_annot = .7
#         col = col2
#         m_size = 3.5
#     elif plot_num == 2:
#         x_annot = 1.09
#         y_annot = .5
#         col = col3
#         m_size = 2.5
#     elif plot_num == 3:
#         x_annot = 1.09
#         y_annot = .3
#         col = col4
#         m_size = 1.5
#     elif plot_num == 4:
#         x_annot = 1.09
#         y_annot = .1
#         col = col5
#         m_size = 1
        



    
#     for i,  arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
#         i_arc = i+1
        
#         auto = obj_m1.Residuals_obs[arc]
#         manu = pd.DataFrame.from_dict(obj_m1.__dict__['OrbitResids'][arc]['resids'])
#         merge_resids = pd.merge(left  = auto , left_on  = 'Date',
#                                 right = manu , right_on = 'Date')

        
#         str_run_param = 'run_parameters'+ arc
#         final_iter = obj_m1.__dict__[str_run_param]['str_iteration']

#         index_pce_x = merge_resids['StatSatConfig'] == 'PCE X    '
#         index_pce_y = merge_resids['StatSatConfig'] == 'PCE Y    '
#         index_pce_z = merge_resids['StatSatConfig'] == 'PCE Z    '

#         fig.add_trace(go.Scattergl(  x=merge_resids['Date'][index_pce_x],
#                                      y=merge_resids['Residual'][index_pce_x],
#                                      name= 'iieout  X',
#                                      mode='markers',
#                                      marker=dict(color=col,
#                                      size=m_size,),
#                                      showlegend=True,
#                                      ),
#                                      row=1, col=1,
#                                      )
#         fig.add_trace(go.Scattergl(  x=merge_resids['Date'],
#                                      y=merge_resids['X'],
#                                      name= 'manual X',
#                                      mode='markers',
#                                      marker=dict(color='red',
#                                      size=m_size-2,),
#                                      showlegend=True,
#                                      ),
#                                      row=1, col=1,
#                                      )
#         fig.add_trace(go.Scattergl(  x=merge_resids['Date'],
#                                      y=merge_resids['Residual'][index_pce_x] - merge_resids['X'],
#                                      name= 'residual of residual X',
#                                      mode='markers',
#                                      marker=dict(color='green',
#                                      size=m_size-2,),
#                                      showlegend=True,
#                                      ),
#                                      row=1, col=2,
#                                      )
#         fig.add_trace(go.Scattergl(x=merge_resids['Date'][index_pce_y],
#                                  y=merge_resids['Residual'][index_pce_y],
#                                  name= 'iieout Y',
#                                  mode='markers',
#                                  marker=dict(color=col,
#                                  size=m_size,),
#                                  showlegend=True,
#                                  ),
#                                  row=2, col=1,
#                                  )
#         fig.add_trace(go.Scattergl(x=merge_resids['Date'],
#                                  y=merge_resids['Y'],
#                                  name= 'manual Y',
#                                  mode='markers',
#                                  marker=dict(color='red',
#                                  size=m_size-2,),
#                                  showlegend=True,
#                                  ),
#                                  row=2, col=1,
#                                  )
#         fig.add_trace(go.Scattergl(  x=merge_resids['Date'],
#                                      y=merge_resids['Residual'][index_pce_y] - merge_resids['Y'],
#                                      name= 'residual of residual Y',
#                                      mode='markers',
#                                      marker=dict(color='green',
#                                      size=m_size-2,),
#                                      showlegend=True,
#                                      ),
#                                      row=2, col=2,
#                                      )
#         fig.add_trace(go.Scattergl(x=merge_resids['Date'][index_pce_z],
#                                  y=merge_resids['Residual'][index_pce_z],
#                                  name= 'iieout Z',
#                                  mode='markers',
#                                  marker=dict(color=col,
#                                  size=m_size,),
#                                  showlegend=True,
#                                  ),
#                                  row=3, col=1,
#                                  )
#         fig.add_trace(go.Scattergl(x=merge_resids['Date'],
#                                  y=merge_resids['Z'],
#                                  name= 'manual Z',
#                                  mode='markers',
#                                  marker=dict(color='red',
#                                  size=m_size-2,),
#                                  showlegend=True,
#                                  ),
#                                  row=3, col=1,
#                                  )
#         fig.add_trace(go.Scattergl(  x=merge_resids['Date'],
#                                      y=merge_resids['Residual'][index_pce_z] - merge_resids['Z'],
#                                      name= 'residual of residual Z',
#                                      mode='markers',
#                                      marker=dict(color='green',
#                                      size=m_size-2,),
#                                      showlegend=True,
#                                      ),
#                                      row=3, col=2,
#                                      )                                 
#     fig.update_layout(
#                     autosize=False,
#                     width=1000,
#                     height=900,
#                     font=dict(size=14),
#                     legend= {'itemsizing': 'constant'})

#     fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=1, col=1)
#     fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=3, col=1)
#     fig.update_xaxes( title="Date", row=3, col=1)
#     fig.update_xaxes( title="Date", row=3, col=2)
#     return(fig)




In [ ]:
# %load_ext autoreload
# %autoreload 2


# fig = make_subplots(
#     rows=3, cols=2,
# #     subplot_titles=(['X','Y','Z']),
#     vertical_spacing = 0.1)

# fig = plot_residuals_manual_and_auto_compare(fig,  Obj_Geodyn['msis2'] , 0 )

# fig.show(config=config)


##  Function: XYZ to NTW Conversion

In [ ]:
# %load_ext autoreload
# %autoreload 2

# # import sys
# # sys.path.insert(0,'/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/')
# # from common_functions          import Convert_cartesian_to_NTW_returnall

# def Convert_cartesian_to_NTW_returnall(state_vector, Tmat_input, PCE_bool = True):
#     '''
#     ###### The Satellite Coordinate System: NTW
#     ###        often used to describe orbital errors, relative positions, 
#                and displacements of satellite orbits. 
    
#     ### The NTW system moves with the satellite.
#     ###     T axis is tangential to the orbit and always points to the velocity vector. 
#     ###     N axis lies in the orbital plane, normal to the velocity vector.
#     ###     W axis is normal to the orbital plane (as in the RSW system)
    
#     ### NOTE: We define in-track or tangential displacements as deviations along
#               the T axis. In-track errors are not the same as along-track variations
#               in the RSW system. One way to remember the distinction is that the 
#               in-track errors are in the direc- tion of the velocity, whereas 
#               along-track variations are simply along the velocity vector.
#     '''

# #     print('state_vector', state_vector)
    
#     r_vec = state_vector[:3]    
#     v_vec = state_vector[-3:]   
#     v_vec_norm = np.linalg.norm(v_vec, ord=2)

#     T_hat = v_vec/v_vec_norm
#     W_hat = (np.cross(r_vec,v_vec)) / np.linalg.norm(np.cross(r_vec,v_vec), ord=2)
#     N_hat = np.cross(T_hat, W_hat)
    
#     ### Transformation matrix
# #     transmat_NTW =  np.transpose(np.array([N_hat, T_hat, W_hat ]))
# #     transmat_NTW =  np.vstack([N_hat, T_hat, W_hat ])
#     if PCE_bool == True:
#         transmat_NTW  =  [[ N_hat[0] , T_hat[0] , W_hat[0] ],
#                           [ N_hat[1] , T_hat[1] , W_hat[1] ],
#                           [ N_hat[2] , T_hat[2] , W_hat[2] ] ]
#     else:
#         transmat_NTW = Tmat_input
    
# #     from scipy.linalg import det
# #     det_transmat_NTW = det(transmat_NTW)
# #     print('transmat_NTW      ',transmat_NTW )
# #     print('det(transmat_NTW) ',det_transmat_NTW )

#     inverse_transmat_NTW = np.linalg.inv(transmat_NTW)

#     r_vec_NTW = np.matmul(inverse_transmat_NTW, r_vec  )
    
#     n = r_vec_NTW[0]
#     t = r_vec_NTW[1]
#     w = r_vec_NTW[2]
    
# #     data_ntw = (n, t, w)
#     return r_vec_NTW, transmat_NTW







### Stats Function

In [ ]:
def STATS_residuals(residuals):
    import numpy as np
    n = np.size(residuals)
    mean = (1/n)*(np.sum(residuals))
    variance = (1/n)*(np.sum(np.square(residuals)))
    rms = np.sqrt(variance)
    rms_about_zero = np.sqrt((n/(n-1))*variance)
    return(mean,rms,rms_about_zero)


### Test conversion: Simple look

In [ ]:
obj_m1 = Obj_Geodyn['msis2']
arc= '2018.313'

# state_vector_pce = np.transpose(np.array([obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['X'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Y'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Z'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Xdot'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Ydot'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Zdot']]))

# NTW_pce  = [Convert_cartesian_to_NTW_returnall(x_pce) for x_pce in state_vector_pce[:1]]

### Test conversion: Magnitudes should match

In [ ]:
# test_xyz = state_vector_pce[:1,:3][0]
# test_ntw =np.array(NTW_pce)[0]

# print('XYZ vec:          ',test_xyz)
# print('NTW vec:          ',test_ntw)

# print('magnitude(XYZ):   ', np.linalg.norm(test_xyz)   )
# print('magnitude(NTW):   ', np.linalg.norm(test_ntw)   )



# print('manual mag (XYZ): ', np.sqrt( np.square(test_xyz[0]) + np.square(test_xyz[1]) + np.square(test_xyz[2]) )   )
# print('manual mag (NTW): ', np.sqrt( np.square(test_ntw[0]) + np.square(test_ntw[1]) + np.square(test_ntw[2]) )   )


#### Test for whole array

In [ ]:
# obj_m1 = Obj_Geodyn['msis2']
# arc= '2018.313'

# state_vector_pce = np.transpose(np.array([obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['X'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Y'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Z'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Xdot'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Ydot'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Zdot']]))

# NTW_pce  = [Convert_cartesian_to_NTW_returnall(x_pce) for x_pce in state_vector_pce]

In [ ]:
# NTW_pce[:5]

In [ ]:
# test_mag_xyz = []
# test_mag_ntw = []
# for i ,val in enumerate(NTW_pce):
#     test_xyz = np.array(val)
#     test_ntw = state_vector_pce[i,:3]

#     test_mag_xyz.append(np.linalg.norm(test_xyz))
#     test_mag_ntw.append(np.linalg.norm(test_ntw))

    
    
# test_mag_xyz = np.array(test_mag_xyz)
# test_mag_ntw = np.array(test_mag_ntw)
# #     print('XYZ vec:          ',test_xyz)
# #     print('NTW vec:          ',test_ntw)

# #     print('magnitude(XYZ):   ', np.linalg.norm(test_xyz)   )
# #     print('magnitude(NTW):   ', np.linalg.norm(test_ntw)   )

In [ ]:
# obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Date']

In [ ]:
# test_mag_xyz = []
# test_mag_ntw = []

# pce_df = obj_m1.__dict__['OrbitResids'][arc]['data_PCE']
# for i ,val in enumerate(pce_df['Date'][:5]):
    
#     test_xyz = np.array([pce_df['X'],pce_df['Y'],pce_df['Z']])
#     test_ntw = np.array([pce_df['N'],pce_df['T'],pce_df['W']])

#     test_mag_xyz.append(np.linalg.norm(test_xyz))
#     test_mag_ntw.append(np.linalg.norm(test_ntw))

# print(test_mag_xyz)
# print(test_mag_ntw)


In [ ]:
test_mag_xyz = []
test_mag_ntw = []

pce_df = obj_m1.__dict__['OrbitResids'][arc]['data_PCE']
for i ,val in enumerate(pce_df['Date']):
    
    test_xyz = np.array([pce_df['X'][i],pce_df['Y'][i],pce_df['Z'][i]])
    test_ntw = np.array([pce_df['N'][i],pce_df['T'][i],pce_df['W'][i]])

    test_mag_xyz.append(np.linalg.norm(test_xyz))
    test_mag_ntw.append(np.linalg.norm(test_ntw))

    
test_mag_xyz =np.array(test_mag_xyz)
test_mag_ntw =np.array(test_mag_ntw)

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(['Compare Magnitudes of XYZ and NTW', 'Difference should be zero' ]),
    vertical_spacing = 0.1)

fig.update_layout(title="PCE Coordinate Trans. Test")

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Date'],
                         y=test_mag_xyz,
                     name='XYZ Magnitude',
                     mode='markers',
                     marker=dict(
                         size=6,
                                ),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=1)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Date'],
                         y=test_mag_ntw,
                     name='NTW Magnitude',
                     mode='markers',
                     marker=dict(
                         size=4,
                                ),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=1)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Date'],
                         y= test_mag_xyz - test_mag_ntw,
                     name='Diff Magnitude',
                     mode='markers',
                     marker=dict(
                         size=4,
                                ),
                     showlegend=False),
                     secondary_y=False,
                     row=2, col=1)

fig.update_yaxes( title="Orbit Magnitude",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Residual (m)"   ,exponentformat= 'power',row=2, col=1)

fig.show(config=config)

In [ ]:
# obj_m1 = Obj_Geodyn['msis2']
# arc= '2018.313'

# state_vector_orb = np.transpose(np.array([obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['X'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Y'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Z'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Xdot'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Ydot'], 
#                                           obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Zdot']]))

# NTW_orb  = [Convert_cartesian_to_NTW_returnall(x_orb) for x_orb in state_vector_orb]



# test_mag_xyz = []
# test_mag_ntw = []
# for i ,val in enumerate(NTW_orb):
#     test_xyz = np.array(val)
#     test_ntw = state_vector_orb[i,:3]

#     test_mag_xyz.append(np.linalg.norm(test_xyz))
#     test_mag_ntw.append(np.linalg.norm(test_ntw))

    
    
# test_mag_xyz = np.array(test_mag_xyz)
# test_mag_ntw = np.array(test_mag_ntw)

test_mag_xyz = []
test_mag_ntw = []

orb_df = obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']
for i ,val in enumerate(orb_df['Date']):
    
    test_xyz = np.array([orb_df['X'][i],orb_df['Y'][i],orb_df['Z'][i]])
    test_ntw = np.array([orb_df['N'][i],orb_df['T'][i],orb_df['W'][i]])

    test_mag_xyz.append(np.linalg.norm(test_xyz))
    test_mag_ntw.append(np.linalg.norm(test_ntw))

    
test_mag_xyz =np.array(test_mag_xyz)
test_mag_ntw =np.array(test_mag_ntw)

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(['Compare Magnitudes of XYZ and NTW', 'Difference b/w magnitudes should be zero' ]),
    vertical_spacing = 0.1)

fig.update_layout(title="Orbfil Coordinate Trans. Test")

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=test_mag_xyz,
                     name='XYZ Magnitude',
                     mode='markers',
                     marker=dict(
                         size=6,
                                ),
                     showlegend=False),
                     secondary_y=False,
                     row=1, col=1)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=test_mag_ntw,
                     name='NTW Magnitude',
                     mode='markers',
                     marker=dict(
                         size=4,
                                ),
                     showlegend=False),
                     secondary_y=False,
                     row=1, col=1)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y= test_mag_xyz - test_mag_ntw,
                     name='Diff Magnitude',
                     mode='markers',
                     marker=dict(
                         size=4,
                                ),
                     showlegend=False),
                     secondary_y=False,
                     row=2, col=1)

fig.update_yaxes( title="Orbit Magnitude",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Residual (m)"   ,exponentformat= 'power',row=2, col=1)

fig.show(config=config)

### Test conversion: Difference of Total Residuals should be zero

#### Residuals of XYZ (PCE - orbfil)

In [ ]:
state_vector_orb = np.transpose(np.array([obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['X'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Y'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Z'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Xdot'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Ydot'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Zdot']]))
state_vector_pce = np.transpose(np.array([obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['X'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Y'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Z'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Xdot'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Ydot'], 
                                          obj_m1.__dict__['OrbitResids'][arc]['data_PCE']['Zdot']]))

### Have to convert PCE first to get the correct/consistent transformation matrix
NTW_pce =  [Convert_cartesian_to_NTW_returnall(x_pce, 0, True) for x_pce in state_vector_pce]
Rvec_ntw_pce = []
Tmat         = []
for vec,matrix in NTW_pce:
    Tmat.append(matrix)
    Rvec_ntw_pce.append(vec)

# NTW_orb  = [Convert_cartesian_to_NTW_returnall(x_orb, Tmat, False) for x_orb in state_vector_orb[:5]]
NTW_orb  = [Convert_cartesian_to_NTW_returnall(x_orb, Tmat_i, False) for x_orb, Tmat_i in zip(state_vector_orb,Tmat)]
Rvec_ntw_orb = []
for vecorb,matrixorb in NTW_orb:
    Rvec_ntw_orb.append(vecorb)



X_resid = []
Y_resid = []
Z_resid = []
N_resid = []
T_resid = []
W_resid = []

xyz_orb_mag = []
xyz_pce_mag = []
ntw_orb_mag = []
ntw_pce_mag = []
xyz_mag_resid = []
ntw_mag_resid = []


for i ,ntw_vec in enumerate(NTW_orb):
    x_orb = float(state_vector_orb[i,0])
    y_orb = float(state_vector_orb[i,1])
    z_orb = float(state_vector_orb[i,2])
    n_orb = float(Rvec_ntw_orb[i][0])
    t_orb = float(Rvec_ntw_orb[i][1])
    w_orb = float(Rvec_ntw_orb[i][2])
    
    x_pce = float(state_vector_pce[i,0])
    y_pce = float(state_vector_pce[i,1])
    z_pce = float(state_vector_pce[i,2])
    n_pce = float(Rvec_ntw_pce[i][0])
    t_pce = float(Rvec_ntw_pce[i][1])
    w_pce = float(Rvec_ntw_pce[i][2])

    X_resid.append(x_pce - x_orb)
    Y_resid.append(y_pce - y_orb)
    Z_resid.append(z_pce - z_orb)
    N_resid.append(n_pce - n_orb)
    T_resid.append(t_pce - t_orb)
    W_resid.append(w_pce - w_orb)

#     print('x_pce      ',x_pce)
#     print('x_orb      ',x_orb)
#     print('X_resid    ',x_pce - x_orb)
#     print(' ')
#     print(' ')
#     print(' ')
#     print( '|—————————————————————————————————————————————————————————————————————— |')
#     print(f"| Component |       PCE       |       Orbfil       |       Residual     |")
#     print( '|---------------------------------------------------------------------- |')
#     print(f"|    X      {x_pce:15.4f}      {x_orb:15.4f}    {(x_pce - x_orb):15.4f}     |")
#     print(f"|    Y      {y_pce:15.4f}      {y_orb:15.4f}    {(y_pce - y_orb):15.4f}     |")
#     print(f"|    Z      {z_pce:15.4f}      {z_orb:15.4f}    {(z_pce - z_orb):15.4f}     |")
#     print( '|                                                                       |')
#     print(f"|    N      {n_pce:15.4f}      {n_orb:15.4f}    {(n_pce - n_orb):15.4f}     |")
#     print(f"|    T      {t_pce:15.4f}      {t_orb:15.4f}    {(t_pce - t_orb):15.4f}     |")
#     print(f"|    W      {w_pce:15.4f}      {w_orb:15.4f}    {(w_pce - w_orb):15.4f}     |")
#     print( '|---------------------------------------------------------------------- |')
#     print(f"|   T_mat_xyz_to_ntw          PCE:                                      | \n {Tmat[i]}          ")
# #     print(f"|   det(T_mat_xyz_to_ntw)     PCE:            {NTW_pce[i][4]:24.20f}  |")
#     print( '|---------------------------------------------------------------------- |')
#     print(f"|   T_mat_xyz_to_ntw         Orbfil:                                    | \n {Tmat[i]}          ")
# #     print(f"|   det(T_mat_xyz_to_ntw)    Orbfil:          {NTW_orb[i][4]:24.20f}  |")
#     print( '|———————————————————————————————————————————————————————————————————————|')

    xyz_orb_mag.append(np.linalg.norm(np.array([x_orb,y_orb,z_orb])))
    xyz_pce_mag.append(np.linalg.norm(np.array([x_pce,y_pce,z_pce])))
    ntw_orb_mag.append(np.linalg.norm(np.array([n_orb,t_orb,w_orb])))
    ntw_pce_mag.append(np.linalg.norm(np.array([n_pce,t_pce,w_pce])))

    xyz_mag_resid.append(np.linalg.norm(np.array([x_pce-x_orb , y_pce-y_orb , z_pce - z_orb])))
    ntw_mag_resid.append(np.linalg.norm(np.array([n_pce-n_orb , t_pce-t_orb , w_pce - w_orb])))
    


In [ ]:
# print('xyz_pce_mag ', xyz_pce_mag)
# print('ntw_pce_mag ', ntw_pce_mag)
# print('xyz_orb_mag ', xyz_orb_mag)
# print('ntw_orb_mag ', ntw_orb_mag)
# print('xyz_mag_resid ', xyz_mag_resid)
# print('ntw_mag_resid ', ntw_mag_resid)

In [ ]:
# Transform_xyz_to_ntw     = NTW_orb[i][3]
# resid_ntw                = np.array([n_pce-n_orb , t_pce-t_orb , w_pce - w_orb])
# resid_xyz                =  np.matmul(Transform_xyz_to_ntw  ,  resid_ntw)

# # 
# print(f"    Transform_xyz_to_ntw    \n  {Transform_xyz_to_ntw}")
# print(f"    resid_ntw                 {resid_ntw}")
# print(f"    resid_xyz                 {resid_xyz}")


#### Plot them all again, explicitly

In [ ]:
# print('X_resid',X_resid)
# print('Y_resid',Y_resid)
# print('Z_resid',Z_resid)
# print('N_resid',N_resid)
# print('T_resid',T_resid)
# print('W_resid',W_resid)

# print()

# print('xyz_orb_mag ', xyz_orb_mag)
# print('xyz_pce_mag ', xyz_pce_mag)
# print('ntw_orb_mag ', ntw_orb_mag)
# print('ntw_pce_mag ', ntw_pce_mag)

# print()

# print('xyz_mag_resid ', xyz_mag_resid)
# print('ntw_mag_resid ', ntw_mag_resid)




fig = make_subplots(
    rows=5, cols=2,
    subplot_titles=(['X resid', 'N resid','Y resid','T resid', 'Z resid','W resid', 'XYZ Res. Mag.', 'NTW Res. Mag.','Diff of magnitude resids' ]),
    vertical_spacing = 0.1,
    specs=[[{}, {}],
           [{}, {}],
           [{}, {}],
           [{}, {}],
           [{"colspan": 2}, None]],
)

fig.update_layout(title="Compare Residuals (for all: resid=(pce - orbfil))")

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=X_resid,
                     name='X_resid',
                     mode='markers',
                     marker=dict(size=5,),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=1)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=Y_resid,
                     name='Y_resid',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=2, col=1)

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=Z_resid,
                     name='Z_resid',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=3, col=1)


fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=N_resid,
                     name='N_resid',
                     mode='markers',
                     marker=dict(size=5,),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=2)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=T_resid,
                     name='T_resid',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=2, col=2)

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=W_resid,
                     name='W_resid',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=3, col=2)


#################################################################################


fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=xyz_mag_resid,
                     name='xyz_mag_resid',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=4, col=1)

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=ntw_mag_resid,
                     name='ntw_mag_resid',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=4, col=2)

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=np.array(xyz_mag_resid) - np.array(ntw_mag_resid),
                     name='Diff resid magnitudes',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=5, col=1)


fig.update_yaxes( title="resids [m]",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="resids [m]",exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="resids [m]",exponentformat= 'power',row=3, col=1)
fig.update_yaxes( title="mag of resid",exponentformat= 'power',row=4, col=1)
fig.update_yaxes( title="resids of mag of resid",exponentformat= 'power',row=4, col=1)

fig.update_yaxes( title="",exponentformat= 'power',row=1, col=2)
fig.update_yaxes( title="",exponentformat= 'power',row=2, col=2)
fig.update_yaxes( title="",exponentformat= 'power',row=3, col=2)

fig.update_layout(
                autosize=False,
                width=1100,
                height=1000,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})


fig.show(config=config)


In [ ]:
# print('X_resid',X_resid)
# print('Y_resid',Y_resid)
# print('Z_resid',Z_resid)
# print('N_resid',N_resid)
# print('T_resid',T_resid)
# print('W_resid',W_resid)

# print()

# print('xyz_orb_mag ', xyz_orb_mag)
# print('xyz_pce_mag ', xyz_pce_mag)
# print('ntw_orb_mag ', ntw_orb_mag)
# print('ntw_pce_mag ', ntw_pce_mag)

# print()

# print('xyz_mag_resid ', xyz_mag_resid)
# print('ntw_mag_resid ', ntw_mag_resid)




fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['Diff. of  Magnitude XYZ residual and Magnitude NTW residual' ]),
    vertical_spacing = 0.15)

fig.update_layout(title="Compare Residuals (for all: resid=(pce - orbfil))")


#################################################################################


fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=np.array(xyz_mag_resid) - np.array(ntw_mag_resid),
                     name='ntw_mag_resid',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=1)


fig.update_yaxes( title="resids of magnitudes of residuals [m]",exponentformat= 'power',row=1, col=1)
# fig.update_yaxes( title="resids [m]",exponentformat= 'power',row=2, col=1)
# fig.update_yaxes( title="resids [m]",exponentformat= 'power',row=3, col=1)
# fig.update_yaxes( title="mag of resid",exponentformat= 'power',row=4, col=1)

fig.update_yaxes( title="",exponentformat= 'power',row=1, col=2)
fig.update_yaxes( title="",exponentformat= 'power',row=2, col=2)
fig.update_yaxes( title="",exponentformat= 'power',row=3, col=2)


# fig.update_layout(
#                 autosize=False,
#                 width=1100,
#                 height=900,
#                 font=dict(size=14),
#                 legend= {'itemsizing': 'constant'})


fig.show(config=config)


In [ ]:
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(['Orbfil Mags (XYZ and NTW)', 'Orbfil Diff (|xyz|-|ntw|)','PCE Mags (XYZ and NTW)','PCE Diff (|xyz|-|ntw|)' ]),
    vertical_spacing = 0.2)

fig.update_layout(title="Compare Magnitudes!")

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=xyz_orb_mag,
                     name='xyz_orb_mag',
                     mode='markers',
                     marker=dict(size=5,),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=1)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=ntw_orb_mag,
                     name='ntw_orb_mag',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=1)

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=np.array(xyz_orb_mag)-np.array(ntw_orb_mag),
                     name='diff_ORB',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=2)


fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=xyz_pce_mag,
                     name='xyz_pce_mag',
                     mode='markers',
                     marker=dict(size=5,),
                     showlegend=True),
                     secondary_y=False,
                     row=2, col=1)
fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=ntw_pce_mag,
                     name='ntw_pce_mag',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=2, col=1)

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=np.array(xyz_pce_mag)-np.array(ntw_pce_mag),
                     name='diff_PCE',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=2, col=2)




fig.update_yaxes( title="Orbit Magnitude",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Orbit Magnitude",exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="diff",exponentformat= 'power',row=1, col=2)
fig.update_yaxes( title="diff",exponentformat= 'power',row=2, col=2)
# fig.update_yaxes( title="diff",exponentformat= 'power',row=2, col=2)
# fig.update_yaxes( title="Residual (m)"   ,exponentformat= 'power',row=2, col=1)

fig.show(config=config)

In [ ]:
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(['XYZ: |PCE|- |Orbfil|','NTW: |PCE|- |Orbfil|' ]),
    vertical_spacing = 0.2)

fig.update_layout(title="Residual of Magnitudes")


fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=np.array(xyz_pce_mag) - np.array(xyz_orb_mag) ,
                     name='xyz_pce_mag - xyz_orb_mag',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=1, col=1)

fig.add_trace(go.Scattergl(
                         x=obj_m1.__dict__['OrbitResids'][arc]['data_orbfil']['Date'],
                         y=np.array(ntw_pce_mag) - np.array(ntw_orb_mag) ,
                     name='ntw_pce_mag - ntw_orb_mag',
                     mode='markers',
                     marker=dict(size=3,),
                     showlegend=True),
                     secondary_y=False,
                     row=2, col=1)

fig.update_yaxes( title="XYZ magnitude residuals",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="NTW magnitude residuals",exponentformat= 'power',row=2, col=1)


In [ ]:
# print(mag_resid_ntw)
# print(mag_resid_xyz)

In [ ]:
sys.exit(0)